In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('../data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [22]:
df_last_15_days = df.loc[df['timestamp'] > pd.to_datetime('2018-05-13')]
df_last_15_days = df_last_15_days.loc[(df_last_15_days['event']=='checkout') & 
                                          (df_last_15_days['event']=='viewed product'), :]

In [23]:
grouped = df_last_15_days.groupby(['person', 'model']).agg({'timestamp':'count'})

In [24]:
grouped.reset_index(inplace=True)
grouped.columns = ['person', 'model', 'events_on_model']

In [25]:
grouped = grouped.groupby(['person']).agg({'events_on_model': 'max'})

In [26]:
grouped.to_csv('../data/cant_,max_eventos_sobre_un_mismo_telefono_ult_quincena.csv')